<a href="https://colab.research.google.com/github/mshahmeer1/Radiological-phenotypes-of-ARDS/blob/main/Dimensionality_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import os
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn import metrics

pd.set_option('display.max_colwidth',100000) #https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

In [ ]:
embeddings = pd.read_pickle('embeddings.pkl')
embeddings = embeddings.drop(['image/format'], 
               axis=1)

print(len(embeddings['embedding']))
print(type(embeddings['embedding']))
print(type(embeddings['embedding'].iloc[2]))
print(embeddings['embedding'].iloc[2].size)
print(embeddings['Len'].nunique())

embeddings['dicom']= embeddings['image/id'].str[61:-4]

In [ ]:
array = embeddings['embedding'].tolist()
array_of_arrays= np.stack(array , axis=0)

In [ ]:
# Import PCA
from sklearn.decomposition import PCA

# Create a PCA model with 2 components: pca
pca = PCA(n_components=62)

# Fit the PCA instance to the scaled samples
pca.fit(array_of_arrays)

# Transform the scaled samples: pca_features
pca_features = pca.transform(array_of_arrays)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [ ]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn import metrics

silhouette_score_threshold = 0.45
current_score = 0

while current_score < silhouette_score_threshold:
    # Create a TSNE instance
    model = TSNE(learning_rate=250, perplexity=120)

    # Apply fit_transform to samples
    tsne_features = model.fit_transform(pca_features)

    # Apply KMeans clustering
    kmeans_model = KMeans(n_clusters=6)
    kmeans_model.fit(tsne_features)
    labels = kmeans_model.predict(tsne_features)

    # Calculate silhouette score
    current_score = metrics.silhouette_score(tsne_features, labels)

    # Print current score
    print(f"Current silhouette score: {current_score:.3f}")

# When threshold is met, print the silhouette score and assign the cluster labels to a variable
print(f"Silhouette Coefficient: {metrics.silhouette_score(tsne_features, labels):.3f}")
kmeans_labels_c6 = labels
centroid = kmeans_model.cluster_centers_

In [ ]:
  # Select the 0th feature: xs
xs = tsne_features[:,0]

# Select the 1st feature: ys
ys = tsne_features[:,1]

In [ ]:

kmeans_labels = pd.DataFrame({'image/id': embeddings['image/id'], 'label':kmeans_labels_c6, 'dicom': embeddings['dicom'], 'tsne_features_xs':xs, 'tsne_features_ys':ys})

kmeans_labels.to_pickle('kmeans_labels450')

In [ ]:
from google.colab import files
files.download('kmeans_labels450')

In [ ]:
print(centroid)

In [ ]:
xs = tsne_features[:,0]

ys = tsne_features[:,1]

plt.scatter(xs,ys, marker ='.', alpha = 0.5, s=10, c=labels)
# plt.savefig('Downloads/kmeans_on_tsne_features_0.450', format='png', dpi=1200)
plt.show()

In [ ]:
y = 0

def cluster(x):
    while x<0.450:
        model = TSNE(learning_rate=250, perplexity=120)

        # Apply fit_transform to samples: tsne_features
        tsne_features = model.fit_transform(pca_features)

        # Select the 0th feature: xs
        xs = tsne_features[:,0]

        # Select the 1st feature: ys
        ys = tsne_features[:,1]

        model = KMeans(n_clusters=6)
        model.fit(tsne_features)
        label = model.predict(tsne_features)

        x = metrics.silhouette_score(tsne_features, label)
    
    else:
        print(f"Silhouette Coefficient: {metrics.silhouette_score(tsne_features, label):.3f}")
        kmeans_labels_c6 = label
cluster(y)